In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import AutoModel, BertTokenizerFast
import numpy as np
from sklearn.model_selection import train_test_split
import re
import gensim
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Fake News Prediction

/content/drive/MyDrive/Fake News Prediction


In [ ]:
true_df = pd.read_csv('true_news.csv')
fake_df = pd.read_csv('fake_news.csv')

In [ ]:
true_df['class'] = 1
fake_df['class'] = 0
true_df['title'] = true_df['title'].apply(lambda x: str(x).lower())
fake_df['title'] = fake_df['title'].apply(lambda x: str(x).lower())

In [ ]:
true_df = true_df[['title','class']]
fake_df = fake_df[['title','class']]

In [ ]:
data = true_df.append(fake_df).sample(frac=1).reset_index().drop(columns=['index'])
data.head()

<ipython-input-11-7d734ecda7ce>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = true_df.append(fake_df).sample(frac=1).reset_index().drop(columns=['index'])


,title,class
0,watch: robin williams calls out hypocrisy of a...,0
1,why is obama disarming cops in america at same...,0
2,cafe owner reacts in awesome way after town to...,0
3,(video) ted cruz protesters get all wet when r...,0
4,this bible study leader’s crimes against a 14...,0


In [ ]:

# Preprocessing data
data['title'] = data['title'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x.lower()))
X = [text.split() for text in data['title'].to_list()]  # Tokenize text
y = data['class'].values  # Assuming the target variable is named 'label'

In [ ]:
# Parameters
MAX_LENGTH = 15
BATCH_SIZE = 32
LSTM_HIDDEN_DIM = 128
OUTPUT_DIM = 2  # Assuming binary classification
EPOCHS = 2

# Load BERT model and tokenizer
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


In [ ]:
# Tokenize and encode sequences
tokens = tokenizer.batch_encode_plus(
    data['title'].tolist(),
    max_length=MAX_LENGTH,
    padding='max_length',
    truncation=True,
    return_tensors="pt"
)

# Create dataset
dataset = TensorDataset(tokens['input_ids'], tokens['attention_mask'], torch.tensor(y))

In [ ]:
# Split dataset into training and validation
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Define the combined model
class CombinedModel(nn.Module):
    def __init__(self, bert_model, lstm_hidden_dim, output_dim):
        super(CombinedModel, self).__init__()
        self.bert = bert_model
        self.lstm = nn.LSTM(input_size=768, hidden_size=lstm_hidden_dim, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_dim, output_dim)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_embeddings = bert_output['last_hidden_state']
        lstm_output, _ = self.lstm(bert_embeddings)
        lstm_output = lstm_output[:, -1, :]  # Using the last hidden state
        output = self.fc(lstm_output)
        return output


In [ ]:
# Initialize the combined model
model = CombinedModel(bert, LSTM_HIDDEN_DIM, OUTPUT_DIM)

# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_dataloader):
        batch = [item.to(torch.device('cpu')) for item in batch]  # Ensure data is on the correct device
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_dataloader)}")

# Model evaluation, prediction, etc., would follow


Epoch 1/2, Loss: 0.4822102951706365
Epoch 2/2, Loss: 0.2700451666324391


In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    total_accuracy = 0
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():
        for batch in dataloader:
            batch = [item.to(torch.device('cpu')) for item in batch]
            input_ids, attention_mask, labels = batch

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            # Calculate accuracy
            predictions = torch.argmax(outputs, dim=1)
            total_accuracy += (predictions == labels).sum().item()

    average_loss = total_loss / len(dataloader)
    average_accuracy = total_accuracy / len(dataloader.dataset)
    return average_loss, average_accuracy

# Model evaluation
val_loss, val_accuracy = evaluate(model, val_dataloader)
print(f'Validation Loss: {val_loss:.3f}, Validation Accuracy: {val_accuracy:.3f}')

Validation Loss: 0.242, Validation Accuracy: 0.906


In [ ]:
# Prediction function
def predict(model, tokenizer, text, max_length=MAX_LENGTH):
    model.eval()  # Ensure model is in evaluation mode
    tokens = tokenizer.encode_plus(text, max_length=max_length, truncation=True, padding='max_length', return_tensors='pt')
    input_ids = tokens['input_ids'].to(torch.device('cpu'))
    attention_mask = tokens['attention_mask'].to(torch.device('cpu'))

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
    predictions = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    return predictions.item()  # Return the predicted class index

In [ ]:
# Example prediction
sample_text = "As part of the 2016 Indian banknote demonetisation, India introduced a new 2,000-rupee currency note.."
predicted_class = predict(model, tokenizer, sample_text)
print(f'Predicted class: {predicted_class}')

Predicted class: 1
